In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - 1_tng_analog_sample.ipynb
# AUTHOR - James Lane
# PROJECT - tng-dfs
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''First look at the IllustrisTNG analog sample.
'''

__author__ = "James Lane"

In [ ]:
# %load ../../src/nb_modules/nb_imports.txt
### Imports

## Basic
import numpy as np
import sys, os

## Matplotlib
from matplotlib import pyplot as plt

## Project-specific
src_path = 'src/'
while True:
    if os.path.exists(src_path): break
    if os.path.realpath(src_path).split('/')[-1] in ['tng-dfs','/']:
            raise FileNotFoundError('Failed to find src/ directory.')
    src_path = os.path.join('..',src_path)
sys.path.insert(0,src_path)
from tng_dfs import util as putil

### Notebook setup

%matplotlib inline
plt.style.use(os.path.join(src_path,'mpl/project.mplstyle')) # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Keywords, loading, pathing

In [ ]:
# %load ../../src/nb_modules/nb_setup.txt
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','MW_ANALOG_DIR','FIG_DIR_BASE','FITTING_DIR_BASE',
            'RO','VO','ZO','LITTLE_H','MW_MASS_RANGE']
data_dir,mw_analog_dir,fig_dir_base,fitting_dir_base,ro,vo,zo,h,\
    mw_mass_range = putil.parse_config_dict(cdict,keywords)

# MW Analog 
mwsubs,mwsubs_vars = putil.prepare_mwsubs(mw_analog_dir,h=h,
    mw_mass_range=mw_mass_range,return_vars=True,force_mwsubs=False)

# Figure path
local_fig_dir = './fig/'
fig_dir = os.path.join(fig_dir_base, 
    'notebooks/1_analog_sample/1_tng_analog_sample/')
os.makedirs(local_fig_dir,exist_ok=True)
os.makedirs(fig_dir,exist_ok=True)
show_plots = False

# # Load tree data
# tree_primary_filename = os.path.join(mw_analog_dir,
#     'major_mergers/tree_primaries.pkl')
# with open(tree_primary_filename,'rb') as handle: 
#     tree_primaries = pickle.load(handle)
# tree_major_mergers_filename = os.path.join(mw_analog_dir,
#     'major_mergers/tree_major_mergers.pkl')
# with open(tree_major_mergers_filename,'rb') as handle:
#     tree_major_mergers = pickle.load(handle)
# n_mw = len(tree_primaries)

### Fundamental properties

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

mass_stars = putil.mass_code_to_physical(mwsubs['mass_stars'])

ax.hist(mass_stars, bins=10, histtype='step', color='k', lw=2, density=False)

ax.set_xlabel(r'$M_{\star}$ [$M_{\odot}$]')
ax.set_ylabel(r'$N$')

fig.tight_layout()
fig.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

mass_dm = putil.mass_code_to_physical(mwsubs['mass_dm'])
print('minimum DM mass ',round(mass_dm.min()/1e12,4),
      '(e12 Msun) maximum DM mass ',round(mass_dm.max()/1e12,4),
      '(e12 Msun)')

ax.hist(mass_dm, bins=10, histtype='step', color='k', lw=2, density=False)

ax.set_xlabel(r'$M_{\rm DM}$ [$M_{\odot}$]')
ax.set_ylabel(r'$N$')

fig.tight_layout()
fig.show()

print('Least massive DM masses: ', np.sort(mass_dm)[:4]/1e12,4, 'e12 Msun')
print('Most massive DM masses: ', np.sort(mass_dm)[-4:]/1e12,4, 'e12 Msun')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

hmr_stars = putil.distance_code_to_physical(mwsubs['halfmassrad_stars'])

ax.hist(hmr_stars, bins=10, histtype='step', color='k', lw=2, density=False)

ax.set_xlabel(r'$R_{1/2,\star}$ [kpc]')
ax.set_ylabel(r'$N$')

fig.tight_layout()
fig.show()


## Download Sublink Trees

In [ ]:
force_download_tree = True
tree_dir = data_dir+'mw_analogs/sublink_trees/full/'
for i in range(len(mwsubs)):
    z0_sid = mwsubs[i]['id']
    sublink_filename = tree_dir+'sublink_'+str(z0_sid)+'.hdf5'
    if os.path.exists(sublink_filename) and not force_download_tree:
        continue
    print('Downloading sublink tree for subfind ID: '+str(z0_sid))
    putil.get( mwsubs[i]['trees:sublink'], directory=tree_dir, timeout=None )